In [1]:
# from docksuitex import Protein, Ligand, PocketFinder, BatchVinaDocking, BatchAD4Docking
from docksuitex import *
from docksuitex.utils import fetch_pdb, fetch_sdf, view_molecule, parse_vina_log_to_csv, parse_ad4_dlg_to_csv
from docksuitex.batch_docking import *

In [2]:
fetch_pdb(pdbid = ["1HVR", "2TRX"], save_to="proteins")

✅ Downloaded 1HVR.pdb → saved to C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\proteins\1HVR.pdb
✅ Downloaded 2TRX.pdb → saved to C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\proteins\2TRX.pdb


[WindowsPath('C:/Users/MANGAL/Desktop/New folder (2)/DockSuiteX/proteins/1HVR.pdb'),
 WindowsPath('C:/Users/MANGAL/Desktop/New folder (2)/DockSuiteX/proteins/2TRX.pdb')]

In [3]:
batch = BatchProtein(
    inputs="proteins",
    fix_pdb=True,
    remove_heterogens=True,
    add_hydrogens=True,
    remove_water=True,
    add_charges=True,
    preserve_charge_types=None,
)

batch.prepare_all(save_to="prepared_proteins")

Starting preparation of 2 proteins using 12 workers...
Output directory: C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\prepared_proteins
✅ 1HVR.pdb → saved to 1HVR.pdbqt
✅ 2TRX.pdb → saved to 2TRX.pdbqt
Batch processing completed!


[{'file': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\proteins\\1HVR.pdb',
  'status': 'success',
  'pdbqt_path': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\prepared_proteins\\1HVR.pdbqt'},
 {'file': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\proteins\\2TRX.pdb',
  'status': 'success',
  'pdbqt_path': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\prepared_proteins\\2TRX.pdbqt'}]

In [3]:
batch = BatchPocketFinder(inputs="prepared_proteins", cpu=100)

results = batch.run_all(save_to="pockets_output")

Starting pocket finding for 2 proteins...
Using 2 parallel workers, 50 CPUs per worker
Total CPU allocation: 100
Output directory: C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\pockets_output
✅ 2TRX.pdbqt → found 4 pockets, saved to 2TRX_pockets
✅ 1HVR.pdbqt → found 2 pockets, saved to 1HVR_pockets
Batch processing completed!


In [ ]:
results

In [14]:
fetch_sdf(cid =  ["228", "338", "339", "340"], save_to="ligands")

✅ Downloaded 228.sdf → saved to C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\ligands\228.sdf
✅ Downloaded 338.sdf → saved to C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\ligands\338.sdf
✅ Downloaded 339.sdf → saved to C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\ligands\339.sdf
✅ Downloaded 340.sdf → saved to C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\ligands\340.sdf


[WindowsPath('C:/Users/MANGAL/Desktop/New folder (2)/DockSuiteX/ligands/228.sdf'),
 WindowsPath('C:/Users/MANGAL/Desktop/New folder (2)/DockSuiteX/ligands/338.sdf'),
 WindowsPath('C:/Users/MANGAL/Desktop/New folder (2)/DockSuiteX/ligands/339.sdf'),
 WindowsPath('C:/Users/MANGAL/Desktop/New folder (2)/DockSuiteX/ligands/340.sdf')]

In [7]:
batch = BatchLigand(
    inputs="ligands",
    minimize="mmff94",
    remove_water=True,
    add_hydrogens=True,
    add_charges=True,
    max_workers=4
)
batch.prepare_all(save_to="prepared_ligands")

Starting preparation of 4 ligands using 4 workers...
Output directory: C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\prepared_ligands
✅ 339.sdf → saved to 339.pdbqt
✅ 340.sdf → saved to 340.pdbqt
✅ 338.sdf → saved to 338.pdbqt
✅ 228.sdf → saved to 228.pdbqt
Batch processing completed!


[{'file': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\ligands\\339.sdf',
  'status': 'success',
  'pdbqt_path': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\prepared_ligands\\339.pdbqt'},
 {'file': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\ligands\\340.sdf',
  'status': 'success',
  'pdbqt_path': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\prepared_ligands\\340.pdbqt'},
 {'file': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\ligands\\338.sdf',
  'status': 'success',
  'pdbqt_path': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\prepared_ligands\\338.pdbqt'},
 {'file': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\ligands\\228.sdf',
  'status': 'success',
  'pdbqt_path': 'C:\\Users\\MANGAL\\Desktop\\New folder (2)\\DockSuiteX\\prepared_ligands\\228.pdbqt'}]

### Dock each ligand against a single receptor at all the predicted protein pockets


##### Using Vina

In [ ]:
# ligands = list(Path("prepared_ligands").glob("*.pdbqt"))

batch = BatchVinaDocking(
    receptors=results,
    ligands="prepared_ligands",
)

results_vina = batch.run_all(save_to="vina_batch_results")

parse_vina_log_to_csv(results_dir="vina_batch_results", output_csv="vina_batch_results/vina_summary.csv")

##### Using AutoDock4

In [ ]:
batch_ad4 = BatchAD4Docking(
    receptors_with_centers=results,
    ligands="prepared_ligands",
    
)

results_ad4 = batch_ad4.run_all(save_to="ad4_batch_results")

parse_ad4_dlg_to_csv(results_dir="ad4_batch_results", output_csv="ad4_batch_results/ad4_summary.csv")


Starting AutoDock4 docking for 24 tasks using 12 workers...
Output directory: C:\Users\MANGAL\Desktop\New folder (2)\DockSuiteX\ad4_batch_results
